<a name='top'></a>
# US Hate Crime Analysis

## Table of Contents
1. [Background](#background)
2. [Inspiration](#inspiration)
3. [Methodologies](#methodologies)
    - [Data Extraction](#data-extraction)
    - [Data Exploration](#data-exploration)
    - [Data Preprocessing](#data-preprocessing)
    - [Data Visualization](#data-visualization)
4. [Poster](#poster)

<a name='background'></a>
## Background
[`^ back to top`](#top)

[**Hate crime**](https://en.wikipedia.org/wiki/Hate_crime), also known as bias-motivated crime, is a crime motivated by prejudice or intolerance toward an individual's membership (or perceived membership) to a  group. For example, one of the victims of hate-crime was attacked for having dark color skin. Unfortunately, this kind of crime slowly became an everyday thing. Discrimination and racism keep increasing even though the majority of the public is protesting against these constantly. This analysis project will show some of the hate-crime characteristics in the US from 2010 to 2018. The US was chosen because the US government open the reported hate crimes incident to the public. The sources of data are mention below.

<a name='inspiration'></a>
## Inspiration
[`^ back to top`](#top)

1. How US hate crime trends in 2010 to 2018?
2. What are the biases that used in US hate crime incidents?
3. What are the criminal acts committed in US hate crime incidents?

<a name='methodologies'></a>
## Methodologies
[`^ back to top`](#top)

This analysis consist of several steps:

<br>![methodologies](images/methodologies.png)<br>

This analysis will be using three tools:
1. **Python 3.7 in Jupyter Notebook (Anaconda)**, for data extraction and preprocessing.
2. **Excel 365**, for data visualization.
3. **Powerpoint 365**, for creating the report poster.

<a name='data-extraction'></a>
### I. Data Extraction
[`^ back to top`](#top)

Raw data can be gathered from [FBI: Crime Data Explorer](https://crime-data-explorer.fr.cloud.gov/downloads-and-docs). In this analysis, I use preprocessed data 
Data obtained from [United States Hate Crimes (1991-2018)](https://www.kaggle.com/louissebye/united-states-hate-crimes-19912017) that preprocessed and posted by [Louisse Bye](https://www.kaggle.com/louissebye). This data consist of several files:

|File Name|Description|
|:---|:---|
|hate_crime.csv|Tabular data about each reported hate crime incidents|
|HC Readme.docx|Provide further information about technical definitions used in dataset|
|NIBRS_DataDictionary.pdf|Data dictionary to provide definitions/information|

In [11]:
# Library
import numpy as np
import pandas as pd

In [56]:
# Extract Data
hate_crimes = pd.read_csv('data/raw/hate_crime.csv', low_memory=False)

<a name='data-exploration'></a>
### Data Exploration
[`^ back to top`](#)

**HateCrimeTable (hate_crime.csv)** consist of **201.403 rows** and **28 columns**. Below is the list of HateCrimeTable columns and its value type.

In [99]:
print("""HateCrimeTable
------------------------------------
Rows\t:{}
Columns\t:{}
------------------------------------""".format(*hate_crimes.shape))

print("""Columns Data Type
------------------------------------""")
for column, dtype in zip(hate_crimes.columns, list(hate_crimes.dtypes)):
    print("{}".format(column).ljust(25, ' '), end=' ')
    print(dtype)

HateCrimeTable
------------------------------------
Rows	:201403
Columns	:28
------------------------------------
Columns Data Type
------------------------------------
INCIDENT_ID               int64
DATA_YEAR                 int64
ORI                       object
PUB_AGENCY_NAME           object
PUB_AGENCY_UNIT           object
AGENCY_TYPE_NAME          object
STATE_ABBR                object
STATE_NAME                object
DIVISION_NAME             object
REGION_NAME               object
POPULATION_GROUP_CODE     object
POPULATION_GROUP_DESC     object
INCIDENT_DATE             object
ADULT_VICTIM_COUNT        float64
JUVENILE_VICTIM_COUNT     float64
TOTAL_OFFENDER_COUNT      int64
ADULT_OFFENDER_COUNT      float64
JUVENILE_OFFENDER_COUNT   float64
OFFENDER_RACE             object
OFFENDER_ETHNICITY        object
VICTIM_COUNT              int64
OFFENSE_NAME              object
TOTAL_INDIVIDUAL_VICTIMS  float64
LOCATION_NAME             object
BIAS_DESC                 object
VICTI

This report will only use columns that provide information to answer the quesitions from Inspiration section. These columns are:

In [98]:
used_columns = ['INCIDENT_ID', 'DATA_YEAR', 'STATE_NAME', 'REGION_NAME', 'OFFENSE_NAME', 'LOCATION_NAME', 'BIAS_DESC']
hc = hate_crimes[used_columns]

print("""Used Columns
------------------------------------""")
for column, dtype in zip(hc.columns, list(hc.dtypes)):
    print("{}".format(column).ljust(25, ' '), end=' ')
    print(dtype)

Used Columns
------------------------------------
INCIDENT_ID               int64
DATA_YEAR                 int64
STATE_NAME                object
REGION_NAME               object
OFFENSE_NAME              object
LOCATION_NAME             object
BIAS_DESC                 object


Below is one of the sample of data in HateCrimeTable.

In [101]:
sample = hc.iloc[3]
print("""Sample of Data
------------------------------------""")
for column, value in zip(hc.columns, sample):
    print("{}".format(column).ljust(25, ' '), end='')
    print(value)

Sample of Data
------------------------------------
INCIDENT_ID              44
DATA_YEAR                1991
STATE_NAME               Arkansas
REGION_NAME              South
OFFENSE_NAME             Aggravated Assault;Destruction/Damage/Vandalism of Property
LOCATION_NAME            Highway/Road/Alley/Street/Sidewalk
BIAS_DESC                Anti-White


#### Findings
**1. Multi-Values Columns**

Several column can have more than one value (multi-value columns). These columns are:
- **OFFENSE_NAME**
- **LOCATION_NAME**
- **BIAS_DESC**

The reason these columns can have multiple categories are some incident can't be categorized with only one category. For example, if the offender destroy victim's property and also harrassing them, the offender can be judged for two crimes but still in one incident. The offender can also "hate" more than one bias that victim's have. Some incident's location sometimes categorized as more than one category, or happen in several location.

Multi-values columns must be processed before it can be visualized. In this report, the multi-values columns will handled by creating new table for each columns. This approach can make reporting process easier.

**2. Too Many Categories**

Below is the frequency table for multi-values columns:

In [60]:
offense_cat = set()
for offenses in hc['OFFENSE_NAME'].tolist():
    for offense in offenses.split(';'):
        offense_cat.add(offense)

In [61]:
location_cat = set()
for locations in hc['LOCATION_NAME'].tolist():
    for location in locations.split(';'):
        location_cat.add(location)

In [62]:
bias_cat = set()
for biases in hc['BIAS_DESC'].tolist():
    for bias in biases.split(';'):
        bias_cat.add(bias)

In [63]:
print("""Unique Value
------------------------------------
Offense Types : {}
Location Types: {}
Bias Types    : {}
""".format(len(offense_cat), len(location_cat), len(bias_cat)))

Unique Value
------------------------------------
Offense Types : 48
Location Types: 46
Bias Types    : 35



There are a lot of categories in each multi-values columns. To many categories will reduce readability of the report. To handle this problem, each column will be clustered.

<a name='data-preprocessing'></a>
### Data Preprocessing
[`^ back to top`](#top)

**1. Cleaning Data**

This analysis will only using reported incident from 2010 to 2018. Following states won't be included:
- Guam
- Hawaii
- Federal

The reason is their location are hard to visualize with Excel Map Charts.

In [66]:
df = hc[(hc['DATA_YEAR'] >= 2010) & (hc['DATA_YEAR'] <= 2018)]
df = df.drop(df[(df['STATE_NAME']=="Guam") | (df['STATE_NAME']=="Hawaii") | (df['STATE_NAME']=="Federal")
               ].index)
print("""Total Rows: {} ({:.2f}% of the total data)""".format(df.shape[0], (df.shape[0]/hc.shape[0])*100))

Total Rows: 57758 (28.68% of the total data)


**2. Parse Multi-Value Columns**

Multi-value column will be normalized by creating another table. Data diagram for this implementation are shown as follow:

<br>![data-diagram](images/data_diagram.png)</br>

In [69]:
def parseMVC(df, column_name, label):
    value = []
    iid = df.columns.get_loc("INCIDENT_ID")
    cid = df.columns.get_loc(column_name)
    for col in df.values.tolist():
        for v in col[cid].split(';'):
            value.append([col[iid], v])
    tdf = pd.DataFrame(value, columns=['Id', label])
    return tdf

In [78]:
offenses = parseMVC(df, 'OFFENSE_NAME', 'Offense')
locations = parseMVC(df, 'LOCATION_NAME', 'Location')
biases = parseMVC(df, 'BIAS_DESC', 'Bias')

print(offenses.shape)
offense_df.head(3)

(59856, 2)


,Incident_Id,Offense
0,147593,Aggravated Assault
1,147594,Simple Assault
2,147595,Simple Assault


**3. Clustering Categories in Multi-Value Columns**

Category-Cluster mapping can be seen in this [file](visualization.xlsx). Clustering will be used to reduce variety of category. The clustering uses information from many sources as references and will be done manually.

In [84]:
# Offense Name
offense_map = {
    'Aggravated Assault': 'Assault',
    'Animal Cruelty': 'Assault',
    'Arson': 'Assault',
    'Destruction/Damage/Vandalism of Property': 'Assault',
    'Simple Assault': 'Assault',
    'Burglary/Breaking & Entering': 'Burgarly or Robbery',
    'Motor Vehicle Theft': 'Burgarly or Robbery',
    'Pocket-picking': 'Burgarly or Robbery',
    'Purse-snatching': 'Burgarly or Robbery',
    'Robbery': 'Burgarly or Robbery',
    'Shoplifting': 'Burgarly or Robbery',
    'Stolen Property Offenses': 'Burgarly or Robbery',
    'Theft From Building': 'Burgarly or Robbery',
    'Theft From Coin-Operated Machine or Device': 'Burgarly or Robbery',
    'Theft From Motor Vehicle': 'Burgarly or Robbery',
    'Theft of Motor Vehicle Parts or Accessories': 'Burgarly or Robbery',
    'Drug Equipment Violations': 'Drug/Weapon Law Violations',
    'Drug/Narcotic Violations': 'Drug/Weapon Law Violations',
    'Weapon Law Violations': 'Drug/Weapon Law Violations',
    'Human Trafficking, Commercial Sex Acts': 'Human Trafficking or Abduction',
    'Kidnapping/Abduction': 'Human Trafficking or Abduction',
    'Betting/Wagering': 'Illegal Activities, Fraud and Identity Theft',
    'Embezzlement': 'Illegal Activities, Fraud and Identity Theft',
    'Counterfeiting/Forgery': 'Illegal Activities, Fraud and Identity Theft',
    'Bribery': 'Illegal Activities, Fraud and Identity Theft',
    'Impersonation': 'Illegal Activities, Fraud and Identity Theft',
    'Hacking/Computer Invasion': 'Illegal Activities, Fraud and Identity Theft',
    'Wire Fraud': 'Illegal Activities, Fraud and Identity Theft',
    'Welfare Fraud': 'Illegal Activities, Fraud and Identity Theft',
    'Identity Theft': 'Illegal Activities, Fraud and Identity Theft',
    'Credit Card/Automated Teller Machine Fraud': 'Illegal Activities, Fraud and Identity Theft',
    'False Pretenses/Swindle/Confidence Game': 'Illegal Activities, Fraud and Identity Theft',
    'Murder and Nonnegligent Manslaughter': 'Murder or Manslaughter',
    'Negligent Manslaughter': 'Murder or Manslaughter',
    'Extortion/Blackmail': 'Psychological Harassment',
    'Intimidation': 'Psychological Harassment',
    'Assisting or Promoting Prostitution': 'Sexual Harassment or Assault',
    'Fondling': 'Sexual Harassment or Assault',
    'Incest': 'Sexual Harassment or Assault',
    'Pornography/Obscene Material': 'Sexual Harassment or Assault',
    'Prostitution': 'Sexual Harassment or Assault',
    'Purchasing Prostitution': 'Sexual Harassment or Assault',
    'Rape': 'Sexual Harassment or Assault',
    'Sexual Assault With An Object': 'Sexual Harassment or Assault',
    'Sodomy': 'Sexual Harassment or Assault',
    'Statutory Rape': 'Sexual Harassment or Assault',
    'Not Specified': 'Other',
    'All Other Larceny': 'Other',
}

In [85]:
# Location Name
location_map = {
    "School-College/University": "Educational Buildings",
    "School/College": "Educational Buildings",
    "School-Elementary/Secondary": "Educational Buildings",
    "Government/Public Building": "Goverment Buildings",
    "Jail/Prison/Penitentiary/Corrections Facility": "Goverment Buildings",
    "Military Installation": "Goverment Buildings",
    "ATM Separate from Bank": "Highway, Road or Other Open Space",
    "Bank/Savings and Loan": "Highway, Road or Other Open Space",
    "Highway/Road/Alley/Street/Sidewalk": "Highway, Road or Other Open Space",
    "Parking/Drop Lot/Garage": "Highway, Road or Other Open Space",
    "Rest Area": "Highway, Road or Other Open Space",
    "Service/Gas Station": "Highway, Road or Other Open Space",
    "Abandoned/Condemned Structure": "Industrial",
    "Construction Site": "Industrial",
    "Industrial Site": "Industrial",
    "Amusement Park": "Public Places and Recreation Sites",
    "Arena/Stadium/Fairgrounds/Coliseum": "Public Places and Recreation Sites",
    "Camp/Campground": "Public Places and Recreation Sites",
    "Church/Synagogue/Temple/Mosque": "Public Places and Recreation Sites",
    "Field/Woods": "Public Places and Recreation Sites",
    "Lake/Waterway/Beach": "Public Places and Recreation Sites",
    "Park/Playground": "Public Places and Recreation Sites",
    "Community Center": "Residential or Community Center",
    "Daycare Facility": "Residential or Community Center",
    "Residence/Home": "Residential or Community Center",
    "Shelter-Mission/Homeless": "Residential or Community Center",
    "Tribal Lands": "Residential or Community Center",
    "Auto Dealership New/Used": "Store and Commercial Buildings",
    "Bar/Nightclub": "Store and Commercial Buildings",
    "Commercial/Office Building": "Store and Commercial Buildings",
    "Convenience Store": "Store and Commercial Buildings",
    "Cyberspace": "Store and Commercial Buildings",
    "Department/Discount Store": "Store and Commercial Buildings",
    "Drug Store/Doctor's Office/Hospital": "Store and Commercial Buildings",
    "Farm Facility": "Store and Commercial Buildings",
    "Gambling Facility/Casino/Race Track": "Store and Commercial Buildings",
    "Grocery/Supermarket": "Store and Commercial Buildings",
    "Hotel/Motel/Etc.": "Store and Commercial Buildings",
    "Liquor Store": "Store and Commercial Buildings",
    "Rental Storage Facility": "Store and Commercial Buildings",
    "Restaurant": "Store and Commercial Buildings",
    "Shopping Mall": "Store and Commercial Buildings",
    "Specialty Store": "Store and Commercial Buildings",
    "Air/Bus/Train Terminal": "Transportation Terminals",
    "Dock/Wharf/Freight/Modal Terminal": "Transportation Terminals",
    "Other/Unknown": "Other/Unknown",
}

In [86]:
# Bias Desc
bias_map = {
    "Anti-Mental Disability": "Disability",
    "Anti-Physical Disability": "Disability",
    "Anti-Female": "Gender",
    "Anti-Gender Non-Conforming": "Gender Identity",
    "Anti-Heterosexual": "Gender Identity",
    "Anti-Male": "Gender Identity",
    "Anti-Transgender": "Gender Identity",
    "Anti-American Indian or Alaska Native": "Racial or Ethnicity",
    "Anti-Arab": "Racial or Ethnicity",
    "Anti-Asian": "Racial or Ethnicity",
    "Anti-Black or African American": "Racial or Ethnicity",
    "Anti-Hispanic or Latino": "Racial or Ethnicity",
    "Anti-Multiple Races, Group": "Racial or Ethnicity",
    "Anti-Native Hawaiian or Other Pacific Islander": "Racial or Ethnicity",
    "Anti-Other Race/Ethnicity/Ancestry": "Racial or Ethnicity",
    "Anti-White": "Racial or Ethnicity",
    "Anti-Atheism/Agnosticism": "Religion or Belief",
    "Anti-Buddhist": "Religion or Belief",
    "Anti-Catholic": "Religion or Belief",
    "Anti-Eastern Orthodox (Russian, Greek, Other)": "Religion or Belief",
    "Anti-Hindu": "Religion or Belief",
    "Anti-Islamic (Muslim)": "Religion or Belief",
    "Anti-Jehovah's Witness": "Religion or Belief",
    "Anti-Jewish": "Religion or Belief",
    "Anti-Mormon": "Religion or Belief",
    "Anti-Multiple Religions, Group": "Religion or Belief",
    "Anti-Other Christian": "Religion or Belief",
    "Anti-Other Religion": "Religion or Belief",
    "Anti-Protestant": "Religion or Belief",
    "Anti-Sikh": "Religion or Belief",
    "Anti-Bisexual": "Sexual Orientation",
    "Anti-Gay (Male)": "Sexual Orientation",
    "Anti-Lesbian (Female)": "Sexual Orientation",
    "Anti-Lesbian, Gay, Bisexual, or Transgender (Mixed Group)": "Sexual Orientation",
    "Unknown (offender's motivation not known)": "Unknown",
}

In [87]:
def addCluster(df, column_name, cmap):
    tdf = df.copy()
    cluster = [cmap[value] for value in df[column_name].tolist()]
    tdf["Cluster"] = cluster
    return tdf

In [89]:
offense_df = addCluster(offenses, 'Offense', offense_map)
location_df = addCluster(locations, 'Location', location_map)
bias_df = addCluster(biases, 'Bias', bias_map)

print(offense_df.shape)
offense_df.head(3)

(59856, 3)


,Incident_Id,Offense,Cluster
0,147593,Aggravated Assault,Assault
1,147594,Simple Assault,Assault
2,147595,Simple Assault,Assault


**4. Rename, Rearangge and Convert DataFrame to .csv**

This final step intends to make the data easier to analyze. Further analysis will be done on Excel.

In [92]:
incident_columns = ['INCIDENT_ID', 'DATA_YEAR', 'STATE_NAME', 'REGION_NAME']
incident_df = df[incident_columns]
incident_df.columns = ['Incident_Id', 'Year', 'State', 'Region']

print(incident_df.shape)
incident_df.head(3)

(57758, 4)


,Incident_Id,Year,State,Region
143578,147593,2010,Alaska,West
143579,147594,2010,Alaska,West
143580,147595,2010,Alaska,West


In [17]:
incident_df.to_csv('data/incidet.csv', index=False)
offense_df.to_csv('data/offense.csv', index=False)
location_df.to_csv('data/location.csv', index=False)
bias_df.to_csv('data/bias.csv', index=False)

<a name='data-visualization'></a>
### IV. Data Visualization
[`^ back to top`](#top)

There are two visualization from these data: (1) [dashboard](visualization.xlsx); and (2) poster ([ori](media/poster.png); [mini](media/poster_mini.png)). 

<a name='poster'></a>
## Poster
[`^ back to top`](#top)

![poster](images/poster_mini.png)